In [1]:
import pandas as pd

In [7]:
dataset_name = 'sidra_tab1092_bovino'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [6]:
(pd.read_json('https://apisidra.ibge.gov.br/values/t/1092/n3/all/v/285/p/last%2094/c12716/115236/c18/992/c12529/118225')
 .to_json(f'{raiz}/bronze_{dataset_name}_uf.json')
 )

(pd.read_json('https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/285/p/last%2094/c12716/115236/c18/992/c12529/118225')
 .to_json(f'{raiz}/bronze_{dataset_name}_BR.json')
 )

KeyboardInterrupt: 

### Prata

In [8]:
(pd.read_json(f'{raiz}/bronze_{dataset_name}_uf.json')
 .filter(['D1C', 'D3C', 'V'])
 .drop(0, axis = 0)
 .assign(**{'ano': lambda tri: tri['D3C'].str.extract('(\\d{4})').astype(int),
            'peso_kg': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'uf': lambda df: df['D1C'].astype(int),
            'fonte': 'Carcaça Bovina'
            })
 .dropna(subset='peso_kg')
 .groupby(['uf', 'ano', 'fonte'])
 .agg({'peso_kg': sum})
 .to_parquet(f'{raiz}/prata_{dataset_name}.parquet')
 )

In [9]:
df_br = (pd.read_json(f'{raiz}/bronze_{dataset_name}_BR.json')
 .filter(['D1C', 'D3C', 'V'])
 .drop(0, axis = 0)
 .assign(**{'ano': lambda tri: tri['D3C'].str.extract('(\\d{4})').astype(int),
            'peso_kg': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'uf': lambda df: df['D1C'].astype(int),
            'fonte': 'Carcaça Bovina'
            })
 .groupby(['uf', 'ano', 'fonte'])
 .agg({'peso_kg': sum})
 )

### Ouro

Peso total de carcaças dos bovinos abatidos (kg)

Aproveitamento da carcaça: 50%

Cálculo: ((Carcaça Bovina/aproveitamento%) * (Parâmetro_x%))/1000

UF e Brasil = t

NPK na forma de Carne bovina retirada do solo via animal encaminhado para abate (t)

In [3]:
aproveitamento = 50/100

parametro_N = 3/100
parametro_P = 0.8/100
parametro_K = 0.2/100

In [10]:
df = (pd.concat([pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet'), df_br])
      .assign(**{
          'N': lambda df: ((df['peso_kg']/aproveitamento)*parametro_N)/1000,
          'P': lambda df: ((df['peso_kg']/aproveitamento)*parametro_P)/1000,
          'K': lambda df: ((df['peso_kg']/aproveitamento)*parametro_K)/1000
          })
      .reset_index()
      .melt(id_vars=['uf', 'ano', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
      .convert_dtypes()
      .set_index(['uf', 'ano', 'fonte'])
      )

df.to_csv(f'{raiz}/ouro_npk_carcaca_bovina.csv')

df

elemento         valor
uf ano  fonte                                
11 2000 Carcaça Bovina        N    4822.42992
   2001 Carcaça Bovina        N     7652.7219
   2002 Carcaça Bovina        N   12206.90538
   2003 Carcaça Bovina        N   12620.08572
   2004 Carcaça Bovina        N   14316.70368
...                         ...           ...
1  2019 Carcaça Bovina        K  32875.404904
   2020 Carcaça Bovina        K  31299.553208
   2021 Carcaça Bovina        K  29825.044564
   2022 Carcaça Bovina        K  32049.279188
   2023 Carcaça Bovina        K  26020.506492

[1854 rows x 2 columns]